# Multiplexed biosensing in bacteria

We will be dealing with the system


$\dot m_i = \alpha_i \frac{\theta_i^a u_i}{1 + \sum_j \theta_j^a u_j} - \delta_0 m_i$

$\dot p_i = \beta_i \frac{\theta_i^b m_i}{1 + \sum_j \theta_j^b m_j} - \delta p_i -\eta_i \frac{\theta_i^D p_i}{1 + \sum_j \theta_j^D p_j}$,

where $\alpha_i, \beta_i, \delta_0, \delta, \eta_i$ are constants and
$\theta_i^a, \theta_i^b, \theta_i^D$ are design parameters.

### Design problem
We want to guarantee that the steady-state behavior of the system satisfies

1) $p_i \ge p_i^H$ when $u_i \ge u_i^H$.

We will also assume that $p_i$ and $u_i$ are bounded above by $p_i^S$ and $u_i^S$, respectively.

Moreover, we will assume that we also get the bound

2) $m_i^S \ge m_i \ge m_i^H$ when the $u_i$ exceed their thresholds.

Finally, we will assume that

3) the quantities

$B_i = \eta_i \frac{\theta_i^D p_i}{1 + \sum_j \theta_j^D p_j}$

are bounded above by $B_i^S$.

### Writing constraints

We define new variables $e_i = \theta_i^a u_i$, $f_i = \theta_i^b m_i$, $g_i = \theta_i^D p_i$. The constraints become

C3. $\eta_i \frac{g_i}{1 + \sum_j g_j} \le B_i^S$

C2. $\alpha_i \frac{e_i}{1 + \sum_j e_j} \ge \delta_0 m_i^H$

C1. $\beta_i \frac{f_i}{1 + \sum_j f_j} - B_i^S \ge \delta p_i^H$

In [8]:
import numpy as np
from pacti.contracts import PolyhedralIoContract

# number of components
n = 4

# constants
eta = np.ones((n)) * 1
alpha = np.ones((n)) * 1
beta = np.ones((n)) * 1
delta = 1
delta0 = 1

# requirements
u_H = np.ones((n)) * 0.7
u_S = np.ones((n)) * 0.8
p_H = np.ones((n)) * 0.01
p_S = np.ones((n)) * 11.1
m_H = np.ones((n)) * 0.1
m_S = np.ones((n)) * 1
B_S = np.ones((n)) * 0.01

# define helper function to return a string representing the addition of a prefix indexed by a range of numbers
def add_n(prefix, count):
    return '+'.join([prefix+str(i) for i in range(count)])

# top-level constraints
top_level_C3 = PolyhedralIoContract.from_strings(
    input_vars=[],
    output_vars=[f"{letter}_{i}" for letter in ["g"] for i in range(n) ],
    assumptions=[],
    guarantees=
        # constraint 3
        [f"{eta[i]}*g_{i} <= {B_S[i]}*(1 + {add_n('g_', n)})" for i in range(n)]
    )


top_level_C2 = PolyhedralIoContract.from_strings(
    input_vars=[],
    output_vars=[f"{letter}_{i}" for letter in ["e"] for i in range(n) ],
    assumptions=[],
    guarantees=
        # constraint 2
        [f"{alpha[i]}*e_{i} >= {delta0*m_H[i]}*(1 + {add_n('e_', n)})" for i in range(n)]
    )

top_level_C1 = PolyhedralIoContract.from_strings(
    input_vars=[],
    output_vars=[f"{letter}_{i}" for letter in ["f"] for i in range(n) ],
    assumptions=[],
    guarantees=
        # constraint 1
        [f"{beta[i]}*f_{i} >= {delta*p_H[i] + B_S[i]}*(1 + {add_n('f_', n)})" for i in range(n)]
    )

# contracts defining the variables e_i, f_i, g_i
component_e = PolyhedralIoContract.from_strings(
    input_vars=[f"{letter}_{i}" for letter in ["theta_a"] for i in range(n) ],
    output_vars=[f"{letter}_{i}" for letter in ["e"] for i in range(n) ],
    assumptions=[],
    guarantees= 
    [f"{u_H[i]} * theta_a_{i} <= e_{i} <= {u_S[i]} * theta_a_{i}" for i in range(n)]
)

component_f = PolyhedralIoContract.from_strings(
    input_vars=[f"{letter}_{i}" for letter in ["theta_b"] for i in range(n) ],
    output_vars=[f"{letter}_{i}" for letter in ["f"] for i in range(n) ],
    assumptions=[],
    guarantees=
    [f"{m_H[i]} * theta_b_{i} <= f_{i} <= {m_S[i]} * theta_b_{i}" for i in range(n)]
)

component_g = PolyhedralIoContract.from_strings(
    input_vars=[f"{letter}_{i}" for letter in ["theta_D"] for i in range(n) ],
    output_vars=[f"{letter}_{i}" for letter in ["g"] for i in range(n) ],
    assumptions=[],
    guarantees=
    [f"{p_H[i]} * theta_D_{i} <= g_{i} <= {p_S[i]} * theta_D_{i}" for i in range(n)]
)

# compute the quotient
theta_D_contract = top_level_C3.quotient(component_g)
theta_a_contract = top_level_C2.quotient(component_e)
theta_b_contract = top_level_C1.quotient(component_f)


In [9]:
print(theta_D_contract)
print(theta_a_contract)
print(theta_b_contract)

InVars: []
OutVars:[theta_D_0, theta_D_1, theta_D_2, theta_D_3]
A: [
  
]
G: [
  10.99 theta_D_0 - 0.0001 theta_D_1 - 0.0001 theta_D_2 - 0.0001 theta_D_3 <= 0.01
  -0.0001 theta_D_0 + 10.99 theta_D_1 - 0.0001 theta_D_2 - 0.0001 theta_D_3 <= 0.01
  -0.0001 theta_D_0 - 0.0001 theta_D_1 + 10.99 theta_D_2 - 0.0001 theta_D_3 <= 0.01
  -0.0001 theta_D_0 - 0.0001 theta_D_1 - 0.0001 theta_D_2 + 10.99 theta_D_3 <= 0.01
]
InVars: []
OutVars:[theta_a_0, theta_a_1, theta_a_2, theta_a_3]
A: [
  
]
G: [
  -0.63 theta_a_0 + 0.08 theta_a_1 + 0.08 theta_a_2 + 0.08 theta_a_3 <= -0.1
  0.08 theta_a_0 - 0.63 theta_a_1 + 0.08 theta_a_2 + 0.08 theta_a_3 <= -0.1
  0.08 theta_a_0 + 0.08 theta_a_1 - 0.63 theta_a_2 + 0.08 theta_a_3 <= -0.1
  0.08 theta_a_0 + 0.08 theta_a_1 + 0.08 theta_a_2 - 0.63 theta_a_3 <= -0.1
]
InVars: []
OutVars:[theta_b_0, theta_b_1, theta_b_2, theta_b_3]
A: [
  
]
G: [
  -0.098 theta_b_0 + 0.02 theta_b_1 + 0.02 theta_b_2 + 0.02 theta_b_3 <= -0.02
  0.02 theta_b_0 - 0.098 theta_b_1 + 0.0